# Lesson 2.1 - Experiment Tracking with MLflow / W&B

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zenml-io/zenbytes/blob/main/2-1_Experiment_Tracking.ipynb)

***Key Concepts:*** *Experiment Tracking, MLflow, Weights & Biases*

When training models, you often need to run hundreds of experiments with different types of models and different hyperparameters to find what works best. Keeping track of every experiment and how each design decision affected the model performance is hardly feasible without additional tools. That is why experiment trackers like [TensorBoard](https://www.tensorflow.org/tensorboard), [Weights & Biases](https://wandb.ai/site), or [MLflow](https://mlflow.org/) are often one of the first touchpoints ML practitioners have with MLOps as they progress through the ML journey. In addition, these tools are invaluable for larger ML teams, as they allow them to share experiment results and collaborate during experimentation.

Since there are many excellent experiment tracking tools, we should aim to prevent vendor lock-in by writing modular ML code that allows us to switch between different tools easily. That is precisely what ZenML will do for us.

This lesson will focus on how to effectively track experiments with [MLflow](https://mlflow.org/), which is one of the most popular open-source MLOps tools and used by many ML teams in practice.

For research-heavy ML teams, we have also included a short bonus section at the end on how to use [Weights & Biases](https://wandb.ai/site) instead of MLflow in your ZenML pipelines.

Please run the following commands to install both tools with their respective dependencies. This will also restart the kernel of your notebook.

In [ ]:
%pip install "zenml[server]"
!zenml integration install sklearn mlflow wandb -y
!rm -rf .zen
!zenml init
%pip install pyparsing==2.4.2  # required for Colab

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

**Colab Note:** On Colab, you need an [ngrok account](https://dashboard.ngrok.com/signup) to view some of the visualizations later. Please set up an account, then set your user token below:

In [ ]:
NGROK_TOKEN = ""  # TODO: set your ngrok token if you are working on Colab

In [ ]:
from zenml.environment import Environment

if Environment.in_google_colab():  # Colab only setup

    IN_COLAB = True

    # clone zenbytes repo to get source code of previous lessons
    !git clone https://github.com/zenml-io/zenbytes.git  # noqa
    !mv zenbytes/steps .
    !mv zenbytes/pipelines .

    # install ngrok and expose port 8080
    !pip install pyngrok
    !ngrok authtoken {NGROK_TOKEN}

Next, let's import our pipeline definition and some of the pipeline steps that we built in the previous lessons:

In [ ]:
from pipelines.digits_pipeline import digits_pipeline
from steps.evaluator import evaluator
from steps.importer import importer

## Register an MLFlow Experiment Tracker

[MLflow](https://mlflow.org/) is an amazing open-source MLOps platform that provides powerful tools to handle various ML lifecycle steps, such as experiment tracking, code packaging, model deployment, and more. We will focus on the [MLflow Tracking](https://mlflow.org/docs/latest/tracking.html) component in this lesson, but we will learn about other MLflow features in later lessons.

To use MLFlow in a ZenML pipeline, we first need to add MLflow into our ZenML MLOps stack. 
We do this by registering a new experiment tracker with ZenML that we then add into our current stack:

In [ ]:
# Register the MLflow experiment tracker
!zenml experiment-tracker register mlflow_tracker --flavor=mlflow

# Create a new stack that includes an MLflow experiment
!zenml stack register mlflow_exp_tracker_stack -a default -o default -e mlflow_tracker

# Set the new stack as active
!zenml stack set mlflow_exp_tracker_stack


Let's also turn off MLflow warnings:

In [ ]:
from absl import logging as absl_logging
import warnings

warnings.filterwarnings("ignore")
absl_logging.set_verbosity(-10000)

## Use MLFlow in a ZenML Pipeline

To integrate the MLFlow experiment tracker into our previously defined ZenML pipeline, we only need to adjust the `svc_trainer` step. Let us define a new `svc_trainer_mlflow` step in which we use MLflow's [`mlflow.sklearn.autolog()`](https://www.mlflow.org/docs/latest/python_api/mlflow.sklearn.html#mlflow.sklearn.autolog) feature to automatically log all relevant attributes and metrics of our model to MLflow. 

By adding an `experiment_tracker=mlflow_tracker` parameter in the `@step` decorator, ZenML automatically takes care of initializing MLflow.

In [ ]:
import mlflow
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from zenml.steps import step

@step(enable_cache=False, experiment_tracker="mlflow_tracker")
def svc_trainer_mlflow(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train an sklearn SVC classifier and log to MLflow."""
    mlflow.sklearn.autolog()  # log all model hparams and metrics to MLflow
    model = SVC(gamma=1e-3)
    model.fit(X_train, y_train)
    return model

svc_mlflow_pipeline = digits_pipeline(
    importer=importer(),
    trainer=svc_trainer_mlflow(),
    evaluator=evaluator(),
)

Now, let's do the same for our decision tree trainer step:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

@step(enable_cache=False, experiment_tracker="mlflow_tracker")
def tree_trainer_with_mlflow(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train an sklearn decision tree classifier and log to MLflow."""
    mlflow.sklearn.autolog()  # log all model hparams and metrics to MLflow
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    return model

tree_mlflow_pipeline = digits_pipeline(
    importer=importer(),
    trainer=tree_trainer_with_mlflow(),
    evaluator=evaluator(),
)

And that's it, we're all set up! Now all `pipeline.run()` calls will automatically log all hyperparameters and metrics to MLflow. Let's try it out:

In [ ]:
svc_mlflow_pipeline.run(unlisted=True)
tree_mlflow_pipeline.run(unlisted=True)

To compare our runs within the MLflow UI, run the following cell, then open http://127.0.0.1:4997/ in your browser.

In [ ]:
# This will start a serving process for mlflow
#  - if you want to continue in the notebook you need to manually
#  interrupt the kernel

from zenml.environment import Environment
from zenml.integrations.mlflow.mlflow_utils import get_tracking_uri


def open_mlflow_ui(port=4997):

    if Environment.in_google_colab():  # Colab only setup
        from pyngrok import ngrok

        public_url = ngrok.connect(port)
        print(f"\x1b[31mIn Colab, use this URL instead: {public_url}!\x1b[0m")

    !mlflow ui --backend-store-uri="{get_tracking_uri()}" --port={port}


open_mlflow_ui()

The first thing you will see is an overview of all your runs, as shown below:

![MLflow UI](_assets/2-1/mlflow_ui.png)

Click on the `Parameters >` tab on top of the table to see *all* hyperparameters of your model. Now you can see at a glance which model performed best and which hyperparameters changed between different runs. In our case, we can see that the SVC model with `gamma=0.001` achieved the best test accuracy of `0.969`.

If we click on one of the links in the `Start Time` column, we can see additional details of the respective run. In particular, we can find a `model.pkl` file under the `Artifacts` tab, which we could now use to deploy our model in an inference/production environment. In the next lesson, `2-2_Local_Deployment.ipynb`, we will learn how to do this automatically as part of our pipelines with the [MLflow Models](https://mlflow.org/docs/latest/models.html)  component.

If you would like to inspect the MLflow logs of your runs manually, you can find
the logging location using the `experiment_tracker_url` metadata field of the 
trainer step of your pipeline run, e.g.:

In [ ]:
from zenml.post_execution import get_unlisted_runs

pipeline_run = get_unlisted_runs()[0]
step = pipeline_run.get_step("trainer")
experiment_tracker_url = step.metadata["experiment_tracker_url"].value

print(experiment_tracker_url)

## Alternative Tool: Weights & Biases

Of course, MLflow is not the only tool you can use for experiment tracking. The following example will show how we could achieve the same with another experiment tracking tool: [Weights & Biases](https://wandb.ai/site).
This example assumes you already have some familiarity with W&B. In particular, you need a Weights & Biases account, which you can set up for free [here](https://wandb.ai/login?signup=true).

You then need to define the three variables below to authorize yourself in W&B and to tell ZenML which entity/project you want to log to:
- `WANDB_API_KEY`: your API key, which you can retrieve at [https://wandb.ai/authorize](https://wandb.ai/authorize). Make sure never to share this key (in particular, make sure to remove the key before pushing this notebook to any public Git repositories!).
- `WANDB_ENTITY`: the entity (team or user) that owns the project you want to log to. If you are using W&B alone, just use your username here.
- `WANDB_PROJECT`: the name of the W&B project you want to log to. If you have never used W&B before or want to start a new project, simply type the new project name here, e.g., "zenbytes".

In [ ]:
WANDB_API_KEY = None  # TODO: replace this with your W&B API key
WANDB_ENTITY = None  # TODO: replace this with your W&B entity name
WANDB_PROJECT = "zenbytes"  # TODO: replace this with your W&B project name (if you want to log to a specific project)

In [ ]:
# Register the W&B experiment tracker
!zenml experiment-tracker register wandb_tracker --flavor=wandb --api_key={WANDB_API_KEY} --entity={WANDB_ENTITY} --project_name={WANDB_PROJECT}

# Create a new stack that includes a W&B experiment tracker
!zenml stack register wandb_stack -a default -o default -e wandb_tracker

# Set the new stack as active
!zenml stack set wandb_stack

Next, we need to add wandb to our `svc_trainer` step and use that to initialize our ZenML pipeline. The overall setup is the same as for the MLflow example above: we simply add an `@enable_wandb` decorator to our step, and then we can use `wandb` functionality within the step as we wish.

The main difference to the MLflow example before is that W&B has no sklearn autolog functionality. Instead, we need to call `wandb.log(...)` for each value we want to log to Weights & Biases.

Since we also want to log our test score, we need to adjust our `evaluator` step accordingly as well.

Note that, despite wandb being used in different steps within a pipeline, ZenML handles initializing wandb and ensures that the experiment name is the same as the pipeline name and that the experiment run name is the same as the pipeline run name. This establishes a lineage between pipelines in ZenML and experiments in wandb.

In [ ]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
import wandb
from zenml.steps import step

from pipelines.digits_pipeline import digits_pipeline
from steps.importer import importer


@step(enable_cache=False, experiment_tracker="wandb_tracker")
def svc_trainer_wandb(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train an sklearn SVC classifier and log to W&B."""
    gamma = 1e-3
    wandb.log({"gamma": gamma})  # log gamma hparam to wandb
    model = SVC(gamma=gamma)
    model.fit(X_train, y_train)
    return model

@step(enable_cache=False, experiment_tracker="wandb_tracker")
def evaluator_wandb(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin,
) -> float:
    """Calculate the accuracy on the test set and log to W&B."""
    test_acc = model.score(X_test, y_test)
    wandb.log({"test acc": test_acc})  # log test_acc to wandb
    print(f"Test accuracy: {test_acc}")
    return test_acc

svc_wandb_pipeline = digits_pipeline(
    importer=importer(),
    trainer=svc_trainer_wandb(),
    evaluator=evaluator_wandb(),
)

Finally, execute the cell below to run your pipeline with different gamma values.

In [ ]:
svc_wandb_pipeline.run(unlisted=True)

Run the cell below and follow the link to see the run in your Weights & Biases 
project:

In [ ]:
trainer_step = get_unlisted_runs()[0].get_step("trainer")
experiment_tracker_url = trainer_step.metadata["experiment_tracker_url"].value
print(experiment_tracker_url)

For a more detailed example of how to use Weights & Biases experiment tracking in your ZenML pipeline, see the [ZenML wandb_tracking example](https://github.com/zenml-io/zenml/tree/main/examples/wandb_tracking).

In the [next lesson](2-2_Local_Deployment.ipynb), we will learn how to deploy models locally with MLflow.